# MODIS DATA MANIPULATION
### By Landung Setiawan

In [1]:
import gdal,os,numpy as np,rasterio as rio, xarray as xr
import version_information,glob,datetime,subprocess
import pymodis
from pyproj import Proj, transform
from rasterio import crs
from rasterio.errors import CRSError
from rasterio.transform import Affine
from rasterio.warp import (reproject, Resampling, calculate_default_transform, transform_bounds)
from math import ceil
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests

gdal.UseExceptions()

WxPython missing, no GUI enabled


/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%matplotlib inline

In [3]:
# Ran with ioos conda env
%load_ext version_information
%version_information numpy, rasterio, xarray, gdal,pymodis

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 4.2.0 36 generic x86_64 with debian jessie sid
numpy 1.11.0
rasterio 0.35.1
xarray 0.7.2-18-g539f1ed
gdal 1.11.4
pymodis 1.0.2
Fri Jun 10 16:34:45 2016 PDT

## Create Tile List

In [4]:
def createTiles():
    ''' 
    Function to create list of tiles to download.
    In this case, MODIS tiles around lower 48 and central america are downloaded
    Tiles are based on sinusoidal projection by NASA
    
    '''

    h = []
    v = []
    tile = []

    for k in range(7, 15):
        h.append(k)
    for l in range(4, 10):
        v.append(l)
    for i in range(0, len(h) - 1):
        for j in range(0, len(v) - 1):
            if h[i] < 10 and v[i] >= 10:
                tile.append("h0" + str(h[i]) + "v" + str(v[j]))
            elif v[j] < 10 and h[i] >= 10:
                tile.append("h" + str(h[i]) + "v0" + str(v[j]))
            elif h[i] < 10 and v[j] < 10:
                tile.append("h0" + str(h[i]) + "v0" + str(v[j]))
            else:
                tile.append("h" + str(h[i]) + "v" + str(v[j]))
    return tile
tiles = createTiles()
print(tiles)

['h07v04', 'h07v05', 'h07v06', 'h07v07', 'h07v08', 'h08v04', 'h08v05', 'h08v06', 'h08v07', 'h08v08', 'h09v04', 'h09v05', 'h09v06', 'h09v07', 'h09v08', 'h10v04', 'h10v05', 'h10v06', 'h10v07', 'h10v08', 'h11v04', 'h11v05', 'h11v06', 'h11v07', 'h11v08', 'h12v04', 'h12v05', 'h12v06', 'h12v07', 'h12v08', 'h13v04', 'h13v05', 'h13v06', 'h13v07', 'h13v08']


## Download MODIS Data

In [5]:
#date = datetime.date.today()
date = datetime.date(2001,2,1)
today = date.strftime('%Y.%m.%d')
date

datetime.date(2001, 2, 1)

In [6]:
# Variables for data download
if not os.path.exists('data'):
    os.makedirs('data')
dest = "data/" # This directory must already exist BTW
tiles = tiles
day = date
enddate = "2001.01.01" # The download works backward, so that enddate is anterior to day=
product = "MOD13A3.006"

#if not os.path.exists(os.path.join(dest,day)):
#    os.makedirs(os.path.join(dest,day))

#folder = os.path.join(dest,day)

In [ ]:
downloader = pymodis.downmodis.downModis(destinationFolder=dest, 
                                         tiles=tiles, today=day, enddate=enddate, product=product)
downloader

In [ ]:
downloader.connect()
print "Connection Attempts: " + str(downloader.nconnection)

In [ ]:
#get all files to download, for each day of interest
downloads = []
for day in downloader.getListDays():
    print day
    if not os.path.exists(os.path.join(dest,day)):
        os.makedirs(os.path.join(dest,day))
    files = downloader.getFilesList(day)
    #print files
    #make list of all the files 
    for f in files:
        downloads.append((f,day))
numDownload = len(downloads)
print "Files to Download: " + str(numDownload)

for filename, day in downloads:
    print "DL: " + filename
    downloader.downloadFile(filename,os.path.join(dest,day,filename),day)

In [52]:
for i in os.listdir(dest):
    if "tif" in i or "4326" in i or "txt" in i or "log" in i or "merge" in i or "climatology" in i:
        pass
    else:
        print i
        folder = os.path.join(dest,i)
        # Check that the data has been downloaded
        MODIS_files = glob.glob(os.path.join(folder , '*.hdf'))
        print MODIS_files
        
        convert2GTiff(MODIS_files)
        mergeTile(i)

2000.02.01
[]
['data/2000.02.01/2000.02.01.tif', 'data/2000.02.01/MOD13A3.A2000032.h07v05.006.2015138123528.tif', 'data/2000.02.01/MOD13A3.A2000032.h07v06.006.2015138123527.tif', 'data/2000.02.01/MOD13A3.A2000032.h07v07.006.2015138123528.tif', 'data/2000.02.01/MOD13A3.A2000032.h08v04.006.2015138123529.tif', 'data/2000.02.01/MOD13A3.A2000032.h08v05.006.2015138123531.tif', 'data/2000.02.01/MOD13A3.A2000032.h08v06.006.2015138123531.tif', 'data/2000.02.01/MOD13A3.A2000032.h08v07.006.2015138123530.tif', 'data/2000.02.01/MOD13A3.A2000032.h08v08.006.2015138123528.tif', 'data/2000.02.01/MOD13A3.A2000032.h09v04.006.2015138123531.tif', 'data/2000.02.01/MOD13A3.A2000032.h09v05.006.2015138123531.tif', 'data/2000.02.01/MOD13A3.A2000032.h09v06.006.2015138123530.tif', 'data/2000.02.01/MOD13A3.A2000032.h09v07.006.2015138123530.tif', 'data/2000.02.01/MOD13A3.A2000032.h09v08.006.2015138123528.tif', 'data/2000.02.01/MOD13A3.A2000032.h10v04.006.2015138123531.tif', 'data/2000.02.01/MOD13A3.A2000032.h10v05.

## Extract EVI and Convert File to GeoTiffs

In [27]:
def convert2GTiff(MODIS_files):
    for hdf in MODIS_files:
        print(hdf)
        name_list = hdf.split('/')[2].split('.')
        file_name = '{0}.{1}.{2}.{3}.{4}'.format(name_list[0],name_list[1],name_list[2],name_list[3],name_list[4])

        sds = gdal.Open(hdf)
        subdata = sds.GetSubDatasets()

        # QA
        QA_src = gdal.Open(subdata[10][0])
        QA_band = QA_src.GetRasterBand(1)


        in_file = subdata[1][0]

        ds = gdal.Open(in_file)
        band = ds.GetRasterBand(1)

        block_sizes = band.GetBlockSize()
        x_block_size = block_sizes[0]
        y_block_size = block_sizes[1]

        xsize = band.XSize
        ysize = band.YSize


        driver = gdal.GetDriverByName('GTiff')
        dst_ds = driver.Create(os.path.join(folder,"{}.tif".format(file_name)), xsize, ysize, 1, gdal.GDT_Int16)
        dst_ds.SetGeoTransform(ds.GetGeoTransform())
        dst_ds.SetProjection(ds.GetProjection())



        for i in range(0, ysize, y_block_size):
            if i + y_block_size < ysize:
                rows = y_block_size
            else:
                rows = ysize - i
            for j in range(0, xsize, x_block_size):
                if j + x_block_size < xsize:
                    cols = x_block_size
                else:
                    cols = xsize - j

                data = band.ReadAsArray(j, i, cols, rows)
                QA = QA_band.ReadAsArray(j, i, cols, rows)

                # Perform value replacement and drop QA layer
                data[np.logical_and(QA != 0, QA != 1)] = -3000

                dst_ds.GetRasterBand(1).SetNoDataValue(-3000)
                dst_ds.GetRasterBand(1).WriteArray(data,j,i)


        dst_ds = None
        # Close datasets and unallocate arrays
        dst_ds = None
        data = None
        QA = None
        band = None
        QA_band = None

In [ ]:
MODIS_xml = glob.glob(os.path.join(folder , '*.hdf.xml'))
MODIS_txt = glob.glob(os.path.join(folder , '*.txt'))

#if MODIS_xml != []:
#    [os.remove(xml) for xml in MODIS_xml]
#if MODIS_txt != []:
#    [os.remove(txt) for txt in MODIS_txt]
#if MODIS_files != []:
#    [os.remove(files) for files in MODIS_files]


## Merge MODIS

In [30]:
from rasterio.merge import merge as merge_tool

In [34]:
def mergeTile(today):
    Gtiff_files = glob.glob(os.path.join(folder , '*.tif'))
    print Gtiff_files

    merged = os.path.join(dest,'merged')
    if not os.path.exists(merged):
        os.makedirs(merged)

    output = os.path.join(merged,'%s.tif' % (today))
    with rio.Env():
            sources = [rio.open(f) for f in Gtiff_files]
            data, output_transform = merge_tool(sources)

            profile = sources[0].profile
            profile.pop('affine')
            profile['transform'] = output_transform
            profile['height'] = data.shape[1]
            profile['width'] = data.shape[2]-1
            profile['driver'] = 'GTiff'

            print(profile)

            with rio.open(output, 'w', **profile) as dst:
                dst.write(data)

In [ ]:
dest

## Calculate Climatology

In [9]:
merged = os.path.join(dest,'merged')
month_list = []
count = 0
for j in range(1,13):
    dates = [date for date in os.listdir(merged) if "{}.01".format(j) in date]
    month_list.append(dates)
for m in month_list:
    file_loc = [os.path.join(merged,'{}'.format(i)) for i in m]
    print(file_loc)
    calcMean(file_loc,merged,count)
    
    count = count + 1

['data/merged/2001.01.01.tif']


/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: read_band() is deprecated and will be removed by Rasterio 1.0. Please use read() instead.


(6000, 3600)


/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)
/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/rasterio/warp.py:248: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  src_transform = guard_transform(src_transform).to_gdal()
/home/lsetiawan/anaconda2/envs/climatology/lib/python2.7/site-packages/rasterio/warp.py:250: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  dst_transform = guard_transform(dst_transform).to_gdal()


{'count': 1, 'crs': {u'a': 6371007.181, u'lon_0': 0, u'no_defs': True, u'y_0': 0, u'b': 6371007.181, u'proj': u'sinu', u'x_0': 0, u'units': u'm'}, u'interleave': 'band', 'dtype': 'int16', 'affine': Affine(926.6254330549995, 0.0, -12231455.716333,
       0.0, -926.6254330558334, 5559752.598333), 'driver': u'GTiff', 'transform': (-12231455.716333, 926.6254330549995, 0.0, 5559752.598333, 0.0, -926.6254330558334), 'height': 6000, 'width': 3600, u'tiled': False, 'nodata': -3000}
['data/merged/2000.02.01.tif', 'data/merged/2001.02.01.tif']
(6000, 8400)
{'count': 1, 'crs': {u'a': 6371007.181, u'lon_0': 0, u'no_defs': True, u'y_0': 0, u'b': 6371007.181, u'proj': u'sinu', u'x_0': 0, u'units': u'm'}, u'interleave': 'band', 'dtype': 'int16', 'affine': Affine(926.6254330549995, 0.0, -12231455.716333,
       0.0, -926.6254330558334, 5559752.598333), 'driver': u'GTiff', 'transform': (-12231455.716333, 926.6254330549995, 0.0, 5559752.598333, 0.0, -926.6254330558334), 'height': 6000, 'width': 8400, u'

IndexError: list index out of range

In [8]:
def calcMean(file_loc,merged,count):
    climate = os.path.join(dest,'climatology')
    if not os.path.exists(climate):
        os.mkdir(climate)
    out = os.path.join(climate,'EVI_{}.tif'.format(count))
    total_sum = 0
    with rio.Env():
        month = [rio.open(f) for f in file_loc]
        data = [m.read_band(1) for m in month]

        print(data[0].shape)


        raw = [np.where(data[d] == -3000, np.nan, data[d]) for d in range(0,len(data))]
        for r in range(0,len(raw)):
            total_sum = total_sum + raw[r]
        mean = total_sum / len(data)

        profile = month[0].profile
        src_prj = month[0].crs
        src_trans = month[0].transform
        rows = month[0].height
        cols = month[0].width
        profile['nodata'] = -3000

        print(profile)

        with rio.open(out, 'w', **profile) as dst:
            dst_array = np.empty((rows, cols), dtype='int16')
            reproject(
                        # Source parameters
                        source=mean,
                        src_crs=src_prj,
                        src_transform=src_trans,
                        src_nodata = 0,
                        # Destination paramaters
                        destination=dst_array,
                        dst_transform=src_trans,
                        dst_crs=src_prj,
                        dst_nodata=-3000,
                        resampling=0
                    )

            dst.write(dst_array,1)

In [12]:
dst_array

array([[-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       ..., 
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000],
       [-3000, -3000, -3000, ..., -3000, -3000, -3000]], dtype=int16)

## Reproject File

In [37]:
climate = os.path.join(dest,'climatology')
epsg = os.path.join(climate,'4326')
output = os.path.join(epsg,'%s.tif' % ('January'))
for e in os.listdir(climate):
    if ".aux.xml" in e or "4326" in e:
        pass
    else:
        gtiff = os.path.join(climate,'{}'.format(e))
        #print(os.path.join(climate,'{}'.format(e)))
        if not os.path.exists(epsg):
            os.makedirs(epsg)
        output = os.path.join(epsg,'{}'.format(e))
        print(gtiff)
        project2wgs(gtiff)

data/climatology/EVI_0.tif
{'count': 1, 'crs': {u'a': 6371007.181, u'lon_0': 0, u'no_defs': True, u'y_0': 0, u'b': 6371007.181, u'proj': u'sinu', u'x_0': 0, u'units': u'm'}, 'dtype': 'int16', 'affine': Affine(926.6254330549995, 0.0, -12231455.716333,
       0.0, -926.6254330558334, 5559752.598333), 'driver': 'GTiff', 'transform': (-12231455.716333, 926.6254330549995, 0.0, 5559752.598333, 0.0, -926.6254330558334), 'height': 6000, 'width': 3600, 'nodata': -3000.0}
| 0.01, 0.00,-127.83|
| 0.00,-0.01, 50.00|
| 0.00, 0.00, 1.00|
{'count': 1, 'crs': {'units': 'm', 'init': 'epsg:4326'}, 'dtype': 'int16', 'affine': Affine(0.01, 0.0, -127.829404882663,
       0.0, -0.01, 49.9999999955068), 'driver': 'GTiff', 'transform': Affine(0.01, 0.0, -127.829404882663,
       0.0, -0.01, 49.9999999955068), 'height': 4482, 'width': 6878, 'nodata': -3000.0}
data/climatology/EVI_1.tif
{'count': 1, 'crs': {u'a': 6371007.181, u'lon_0': 0, u'no_defs': True, u'y_0': 0, u'b': 6371007.181, u'proj': u'sinu', u'x_0':

In [21]:
def project2wgs(gtiff):
    with rio.Env():
        with rio.open(gtiff) as src:
            l, b, r, t = src.bounds
            out_kwargs = src.meta.copy()
            out_kwargs['driver'] = 'GTiff'

            print(out_kwargs)

            res = (0.01, 0.01)
            dst_crs = crs.from_string('+units=m +init=epsg:4326')

            #dst_width, dst_height = src.width, src.height
            xmin, ymin, xmax, ymax = [-127.8294048826629989,5.1830409679864857,-59.0561278820333229,49.9999999955067977]#transform_bounds(src.crs, dst_crs, *src.bounds)
            dst_transform = Affine(res[0], 0, xmin, 0, -res[1], ymax)
            dst_width = max(int(ceil((xmax - xmin) / res[0])), 1)
            dst_height = max(int(ceil((ymax - ymin) / res[1])), 1)
            print(dst_transform)

            out_kwargs.update({
                    'crs': dst_crs,
                    'transform': dst_transform,
                    'affine': dst_transform,
                    'width': dst_width,
                    'height': dst_height
                })
            dst_shape = (dst_height, dst_width)
            destination = np.empty(dst_shape, rio.int16)

            print(out_kwargs)

            with rio.open(output, 'w', **out_kwargs) as dst:
                reproject(
                    source=rio.band(src, 1),
                    destination=rio.band(dst, 1),
                    src_transform=src.affine,
                    src_crs=src.crs,
                    src_nodata=src.nodata,
                    dst_transform=out_kwargs['transform'],
                    dst_crs=out_kwargs['crs'],
                    dst_nodata=src.nodata,
                    resampling=0,
                    num_threads=2)

## Numpy to Xarray

In [ ]:
with rio.Env():
    with rio.open(output) as da:
        print(da.meta)
        data = da.read(1)       
        data = np.where(data == da.nodata, np.nan, data)
        print(data)
        
        # Get coords
        nx, ny = da.width, da.height
        x0, y0 = da.bounds.left, da.bounds.top
        dx, dy = da.res[0], -da.res[1]

        coords = {'lat': np.arange(start=y0, stop=(y0 + ny * dy), step=dy),
                  'lon': np.arange(start=x0, stop=(x0 + nx * dx), step=dx)}
        
        dims = ('band', 'lat', 'lon')
        coords['band'] = da.indexes
        
        attrs = {}
        for attr_name in ['crs', 'affine', 'proj']:
            try:
                attrs[attr_name] = getattr(da, attr_name)
            except AttributeError:
                pass
        
        ds = xr.DataArray(data, dims=dims, name='raster',coords=coords)

In [ ]:
da.indexes